# 4 Recommendation System

- Author: Jason Truong
- Last Modified: September 19, 2022
- Email: Jasontruong19@gmail.com

# Table of Contents

1. [Objective and Roadmap](#1Objective)  
2. [Preliminary Data Setup](#2Preliminary)   
3. [Content Based Recommendation](#3Content_Recommend)   
    3.1 [Single Product Comparison](#3.1_single_product)  
    3.2 [Save Vectorizer Model](#3.2_save_vectorizer)  
    3.3 [Recommendation Function](#3.3_recommend_function)  
    3.4 [Test Recommendation System](#3.4_test_recommendation)   
    3.5 [Evaluate Recommendation System](#3.5_evaluate_recommendation)  
4. [Conclusion and Future Works](#4Conclusion)  

# 1. Objective<a class ='anchor' id='1Objective'></a>

To use the predicted sentiment from review text and product description to come up with recommendations for users.

# 2. Preliminary Data Setup<a class ='anchor' id='2Preliminary'></a>

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import random


from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

Load in the dataset

In [2]:
meta_df = pd.read_csv('clean_meta.csv')

In [3]:
# Ensure that the description column has no null values.
meta_df['description_0'].fillna('', inplace = True)

# Check results
meta_df

,title,brand,rank,product_id,description_0,category_1,category_2
0,My Fair Pastry (Good Eats Vol. 9),Alton Brown,"370,026",0000143529,Disc 1: Flour Power (Scones; Shortcakes; South...,Movies,NaN
1,"Barefoot Contessa (with Ina Garten), Entertain...",Ina Garten,"342,914",0000143588,Barefoot Contessa Volume 2: On these three dis...,Movies,NaN
2,Rise and Swine (Good Eats Vol. 7),Alton Brown,"351,684",0000143502,Rise and Swine (Good Eats Vol. 7) includes bon...,Movies,NaN
3,The Power of the Cross Joseph Prince,Joseph Prince,"444,474",000073991X,Have failures in your life caused you to feel ...,Genre for Featured Categories,Exercise & Fitness
4,Live in Houston [VHS],Douglas Miller,"1,005,955",000107461X,Track Listings 1. Come On Everybody 2. My Stre...,Movies,NaN
...,...,...,...,...,...,...,...
156476,Verdi: Otello,Sonya Yoncheva,"68,026",B01HJ1INB0,Tony Award-winning director Bartlett Sher prob...,Studio Specials,Sony Pictures Home Entertainment
156477,Mr. Miracle - Ihn schickt der Himmel,NaN,"344,483",B01HJ3E0PQ,Mr. Miracle DVD Region 2 need an all region DV...,Movies,NaN
156478,The President,Misha Gomiashvili,"199,854",B01HJ6R77G,The President and his family rule the land wit...,Independently Distributed,Drama
156479,She.....Who Would Be Pope,Liv Ullmann,"246,494",B01HJCCLOY,"Filmed in 1972 as Pope Joan, Michael Andersons...",Genre for Featured Categories,Action & Adventure


# 3. Content Based Recommendation<a class ='anchor' id='3Content_Recommend'></a>

The first step is to use the descriptions of the different Amazon items, in this case, movies/tv shows to recommend products that are similar.

In [4]:
# Take the description, title and product id from the metadata
rec_df = meta_df[['title','description_0','product_id']].copy()

In [10]:
# Start off with a subset of the data
rec_df_subset = rec_df.iloc[0:50000,:]

# Check results
rec_df_subset

,title,description_0,product_id
0,My Fair Pastry (Good Eats Vol. 9),Disc 1: Flour Power (Scones; Shortcakes; South...,0000143529
1,"Barefoot Contessa (with Ina Garten), Entertain...",Barefoot Contessa Volume 2: On these three dis...,0000143588
2,Rise and Swine (Good Eats Vol. 7),Rise and Swine (Good Eats Vol. 7) includes bon...,0000143502
3,The Power of the Cross Joseph Prince,Have failures in your life caused you to feel ...,000073991X
4,Live in Houston [VHS],Track Listings 1. Come On Everybody 2. My Stre...,000107461X
...,...,...,...
49995,Shirley Temple: America's Sweetheart Collectio...,"Includes Baby Take a Bow, Bright Eyes & Rebecc...",B000FKPDY8
49996,America's Castles - The Grand Resorts,While taverns and inns have been a part of the...,B000FKP22Q
49997,Dragon in Fury,Dragon in Fury movie,B000FKPDUW
49998,"Rin Tin Tin: Double Feature, Vol. 3",Caryl of the mountains: The 1914 silent film w...,B000FKP42O


Turn the description into numeric features with TF-IDF Vectorizer

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the vectorizer
vectorizer = TfidfVectorizer(stop_words = 'english', min_df = 10)

# Fit
vectorizer.fit(rec_df_subset['description_0'])

# Transform the description
TF_matrix = vectorizer.transform(rec_df_subset['description_0'])

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

movie_index = rec_df_subset[rec_df_subset['title'] =='Rise and Swine (Good Eats Vol. 7)'].index

mov_similarities = cosine_similarity(TF_matrix,dense_output = False)

sim_df = pd.DataFrame({'item':rec_df_subset['title'], 
                       'similarities': np.array(mov_similarities[movie_index,:].todense()).squeeze()})

List of the top recommended products based off of similarities

In [17]:
top_recommend = sim_df.sort_values(by = 'similarities', ascending = False)[0:10]

# Check results
top_recommend

,item,similarities
2,Rise and Swine (Good Eats Vol. 7),1.000000
14782,Love Is the Devil,0.220224
30335,Raquel Welch Collection: (One Million Years B....,0.213950
2530,Manly Men III: Time to Man Up,0.208671
25598,Charles Bronson Action Pack: (Kinjite / Messen...,0.203271
28781,The Ultimate Chick Flick Collection: (The Bang...,0.202631
29728,The Sidney Poitier Collection: (For Love of Iv...,0.194004
29703,The Best of Soul Cinema: (Coffy / Cooley High ...,0.190299
31078,Living Christ,0.185110
48634,Red Dawn / Navy Seals,0.183135


**Check the actual descriptions to see if products are similar.**

In [18]:
rec_df_subset['description_0'][14782]

"This British biographical drama probes the life of painter Francis Bacon (1909-1992), critically acclaimed as the outstanding British painter of the latter half of the 20th Century. This unsympathetic portrait of Bacon (Derek Jacobi) begins when George Dyer (Daniel Craig), a small-time criminal from working-class East End environs, drops through a skylight to rob Bacon's studio -- and is ordered into bed by Bacon. The two become a familiar couple at Bacon's hangout, the Colony Room in Soho. Bacon's sexual interests lean toward S&M, but as the cruel Bacon loses interest in Dyer and begins to look elsewhere, the couple splits. Left to his own devices, Dyer turns to drugs and alcohol -- and a tragic suicide. Visual grotesqueries and a trancelike Ryuichi Sakamoto music score capture the essence of Bacon's work (although paintings by Bacon are not seen onscreen here). The film is told in the form of a flashback from Bacon's successful 1971 retrospective at the Grand Palais in Paris to a pe

In [19]:
rec_df_subset.loc[14782,:]

title                                            Love Is the Devil
description_0    This British biographical drama probes the lif...
product_id                                              6305847045
Name: 14782, dtype: object

In [20]:
rec_df_subset['description_0'][2]

'Rise and Swine (Good Eats Vol. 7) includes bonus footage "Ask Alton" and 3 discs. Disc 1: Breakfast Eats 3 (Bacon Vinaigrette with Grilled Radicchio; Scrap Iron Chef\'s Bacon; Honey Mustard Cure; Molasses Black Pepper Cure; Red Pepper Brine; Soy Honey Brine; Omelet; Omelet for a Crowd; Frittata; Blueberry Muffins; And More...) Disc 2: Fruit Fest (It\'s a Wonderful Waldorf; 10 Minute Applesauce; Baker, Baker; Clotted Cream; Macerated Strawberries; Strawberry Pudding; Frozen Strawberries; Banana Ice Cream; Bananas Foster; Fried Plantains; And more...) Disc 3: Manly Eats (Sirloin Steak; Skirt Steak; Pulled Pork; Corn Dogs; Mini Man Burgers; And More...)'

In [21]:
top_recommend.index[1]

14782

In [22]:
sim_df.sort_values(by = 'similarities', ascending = True)

,item,similarities
24999,The Corpse Vanishes,0.000000
29976,The Best Collection: Crime,0.000000
29979,Psycho Santa / Satan Claus,0.000000
29980,Baby Human: Geniuses in Diapers,0.000000
29981,Star Trek Voyager - The Complete First Season,0.000000
...,...,...
25598,Charles Bronson Action Pack: (Kinjite / Messen...,0.203271
2530,Manly Men III: Time to Man Up,0.208671
30335,Raquel Welch Collection: (One Million Years B....,0.213950
14782,Love Is the Devil,0.220224


Find the Cosine similarity for the requested product instead of generating te cosine similarity for all the products.

In [23]:
# Define the vectorizer
vectorizer2 = TfidfVectorizer(stop_words = 'english', min_df = 10)

# Fit
vectorizer2.fit(rec_df['description_0'])


TfidfVectorizer(min_df=10, stop_words='english')

This approach takes a lot of computation power and memory because the cosine similarity between every product is computed. A more efficient way is to compare only the product that is chosen with every other product.

## 3.1 Single Product Comparision<a class ='anchor' id='3.1_single_product'></a>

In [24]:
# Transform the description
TF_matrix = vectorizer2.transform(rec_df['description_0'])

In [25]:
# Determine the index of the movie
movie_index = rec_df_subset[rec_df_subset['title'] =='Rise and Swine (Good Eats Vol. 7)'].index

In [26]:
TF_matrix_product = vectorizer2.transform(rec_df_subset['description_0'][movie_index])

In [27]:
movie_index

Int64Index([2], dtype='int64')

In [28]:
TF_matrix.shape

(156481, 39048)

In [29]:
TF_matrix[movie_index]

<1x39048 sparse matrix of type '<class 'numpy.float64'>'
	with 54 stored elements in Compressed Sparse Row format>

In [30]:
# Check the shape of the transformed description
TF_matrix_product.shape

(1, 39048)

In [31]:
# Find the similarity between the chosen product with every other product in the dataset
mov_similarities = cosine_similarity(TF_matrix,TF_matrix_product, dense_output = False)

# Check output results
mov_similarities

<156481x1 sparse matrix of type '<class 'numpy.float64'>'
	with 55953 stored elements in Compressed Sparse Row format>

In [32]:
# Convert the movie similarities output into a dataframe
single_df = pd.DataFrame({'item':rec_df['title'], 
                       'similarities': np.array(mov_similarities.todense()).squeeze()})

In [33]:
# Check the top 10 most similar products
single_df.sort_values(by = 'similarities', ascending = False).head(10)

,item,similarities
2,Rise and Swine (Good Eats Vol. 7),1.000000
76483,Good Eats with Alton Brown Vol. 12,0.301367
94294,American Eats: Holiday Foods,0.256991
61427,Food Network: Good Eats with Alton Brown - Bre...,0.247336
92830,Good Eats with Alton Brown - Holiday Treats (R...,0.244306
99539,Food Network Takeout Collection DVD - Good Eat...,0.243199
94305,American Eats: Hot Dogs,0.236412
76760,Good Eats: Cupboard Cuisine - Volume 11,0.224773
94298,American Eats,0.213358
95611,Kitchen Wisdom from Good Eats (Good Eats Vol. 20),0.202247


## 3.2 Save the vectorizer model<a class ='anchor' id='3.2_save_vectorizer'></a>

In [34]:
pickle.dump(vectorizer2, open('recommender_vectorizer.pickle','wb'))

In [35]:
file_vectorizer = pickle.load(open('recommender_vectorizer.pickle','rb'))

## 3.3 Top 10 Recommendation Function<a class ='anchor' id='3.3_recommend_function'></a>

Create a function that would take in the item and output the top 10 recommendations.

In [65]:
def rec_system(vectorizer, sentiment, product_list, product_name):
    '''
    
    The purpose of this function is to recommend products based on the predicted
    sentiment and product name.
    
    The inputs to this function includes the vectorizer model, the sentiment of 
    the review text, the list of products, product name to be used for 
    recommending other products.
    
    
    Parameters
    ----------
    vectorizer = sklearn.feature_extraction.text.TfidfVectorizer (Vectorizer model)
    sentiment = int
    product_list = pandas.core.frame.DataFrame
    product_name = str 
    
    
    Returns
    --------
    Recommended Product list
    
    '''
    
    # Transform the description
    TF_matrix = vectorizer.transform(product_list['description_0'])
    
    # Determine the index of the product of recommendation
    product_index = product_list[product_list['title'] == product_name].index
    
    # Get the TF matrix of the required product
    TF_matrix_product = TF_matrix[product_index]
    
    # Determine the similarity between the product and everything else
    product_similarities = cosine_similarity(TF_matrix,TF_matrix_product, dense_output = False)
    
    # 
    single_df = pd.DataFrame({'item':product_list['title'], 
                       'similarities': np.array(product_similarities.todense()).squeeze()})
    
    # Sort the products by similarity
    single_df_sorted = single_df.sort_values(by = 'similarities', ascending = False)
    
    
    # When the sentiment is positive, recommend the top 10 most similar products
    if sentiment == 1:
        recommended_products = single_df_sorted.head(10)
    
    # Recommend 10 products randomly from the top 100 most similar products
    else:
        # Split out the top 100 recommended_products
        top_100 = single_df_sorted.iloc[:100].reset_index(drop = True)
        
        # Randomly sample 10 numbers between 0 and 99
        ran_num = random.sample(range(0, 100), 10)
        
        # Find the index where the ran_num exists in the top 100 recommended products
        top_100_index = top_100.index.isin(ran_num)
        
        # Determine recommended products
        recommended_products = top_100[top_100_index]
        
    return recommended_products.reset_index(drop = True)

## 3.4 Test out recommendation system<a class ='anchor' id='3.4_test_recommendation'></a>

In [66]:
# Test out recommendation system

rec_system(file_vectorizer, 0 ,rec_df,'Rise and Swine (Good Eats Vol. 7)')

,item,similarities
0,American Eats: Holiday Foods,0.256991
1,Love Is the Devil,0.185373
2,"Surf, Turf &amp; A Side (Good Eats Vol. 14)",0.180079
3,Strawberry Shortcake - Get Well Adventure,0.144364
4,Dare To Cook: Barbecue &amp; Grilling,0.139608
5,Floyd Mayweather jr. Boxing DVD Collection,0.131203
6,Untold Secrets of the Civil War/American India...,0.125405
7,Partying 101: (Bio Dome / P.C.U. / Back to Sch...,0.124416
8,"Strawberry Shortcake - Berry, Merry Christmas",0.123068
9,Elvis Presley MGM Movie Legends Collection: (C...,0.121558


## 3.5 Evaluate recommendation system<a class ='anchor' id='3.5_evaluate_recommendation'></a>

Sample tests can be a movie review + the rating -> Feed into model, Output top 10 movies the person may like.

Use reviews and movie descriptions to determine which movies to recommend based off of if the person rated the movie highly or not.

# 4. Conclusion and Future Works<a class ='anchor' id='4Conclusion'></a>